In [61]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.metrics.cluster import normalized_mutual_info_score
from scipy.fftpack import fft, ifft
from sklearn.decomposition import PCA
import warnings
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle

def main(cgm,cgmpatient,insulin,insulinpatient):
    ## Read CGM Data
    cgmdata=pd.read_csv(cgm)
    ## Read CGM Patient Data
    cgmpatient=pd.read_csv(cgmpatient)
    ## Read Insulind data 
    insulindata=pd.read_csv(insulin)
    ## Read Insulin Patient data
    insulinpatient=pd.read_csv(insulinpatient)
   
    ##Subset CGM
    insulin=insulindata[["Index","Date","Time","BWZ Carb Input (grams)"]]
    
    #Merge date time
    insulin['dt']=pd.to_datetime(insulin['Date'] + ' ' + insulin['Time'])
    #Rename column
    insulin = insulin.rename(columns={'BWZ Carb Input (grams)': 'carbinput'})
    #Subset carb input >0
    insulin_new=insulin[insulin["carbinput"] >0]
    insulin_new['tm2'] = pd.DatetimeIndex(insulin_new['dt'])
    #subset columns
    insulin_join=insulin_new[["dt","carbinput","tm2"]]
    #sort by date time
    insulin_sort=insulin_join.sort_values(by='dt')
    # Choose start time
    insulin_sort['lagtime'] = (insulin_sort['dt'].shift(-1))
    insulin_sort['diff']=insulin_sort['lagtime']-insulin_sort['dt']
    insulin_sort=insulin_sort[insulin_sort['diff'] >='02:00:00']

    ##Read,Subset and sort CGM
    cgm=cgmdata[["Index","Date","Time","Sensor Glucose (mg/dL)"]]
    cgm['dt']=pd.to_datetime(cgm['Date'] + ' ' + cgm['Time'])
    cgm_sort=cgm.sort_values(by='dt')
    
    #Merge cgm_sort and insulin_sort
    new=pd.merge_asof(
    cgm_sort,
    insulin_sort,
   # by="component",
    right_on="dt",
    left_on="dt",
    direction='nearest',
    tolerance=pd.Timedelta('2.5 minute'))
    
    new_fin=new[new["carbinput"]>0]
    new_fin['tm']=pd.to_datetime(new_fin['Date'] + ' ' + new_fin['Time'])
    new_fin['tm2meal'] = pd.DatetimeIndex(new_fin['tm']) + timedelta(hours=2,minutes=0)
    new_fin['tm30meal'] = pd.DatetimeIndex(new_fin['tm']) - timedelta(hours=0,minutes=30)
    new_fin['tm2nomeal'] = pd.DatetimeIndex(new_fin['tm2meal']) + timedelta(hours=2,minutes=0)
    new_fin=new_fin[["dt","tm","tm2meal","tm30meal","tm2nomeal"]]
    
    #merge new with meal dates
    newfinal=pd.merge_asof(
    new,
    new_fin,
   # by="component",
    right_on="dt",
    left_on="dt",
    direction='nearest',
    tolerance=pd.Timedelta('1 minute'))
    
    ####### MEAL DATA EXTRACTION ##########
    newsubset=newfinal[newfinal["carbinput"]>0]
    mealtime=newsubset[["tm30meal"]]
    mealtime["dt"]=mealtime["tm30meal"]
    
    newfinal2=pd.merge_asof(
    newfinal,
    mealtime,
   # by="component",
    right_on="dt",
    left_on="dt",
    tolerance=pd.Timedelta('30 minute'))
    
    mealtime2=newsubset[["tm","tm2meal"]]
    mealtime2["dt"]=mealtime2["tm"]
    
    newfinal3=pd.merge_asof(
    newfinal2,
    mealtime2,
   # by="component",
    right_on="dt",
    left_on="dt",
    tolerance=pd.Timedelta('120 minute'))
    
    Mergedfinal=newfinal3[(newfinal3["tm30meal_y"].notna()) | (newfinal3["tm2meal_y"].notna())]
    mealdata=Mergedfinal[["Sensor Glucose (mg/dL)"]]
    mealdata = mealdata.reset_index()
    mealdata=mealdata[["Sensor Glucose (mg/dL)"]]
    #Imput missing observations with mean value
    mealdata['Sensor Glucose (mg/dL)'].fillna(mealdata['Sensor Glucose (mg/dL)'].mean(), inplace = True)
    
    
    seq= cycle([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])
    mealdata['x'] = [next(seq) for count in range(mealdata.shape[0])]
    mealdata['y']=0
    mealdata['y'] = (mealdata["x"] %30==0).shift(1).cumsum() + 1
    mealdata['y'].fillna(value=1, inplace = True)
    
    meal_data=mealdata.pivot(index='y',columns='x', values='Sensor Glucose (mg/dL)').add_prefix('X').reset_index()
    
    ####### NO MEAL DATA EXTRACTION ##########
    mealtime3=newsubset[["tm","tm2nomeal"]]
    mealtime3["dt"]=mealtime3["tm2nomeal"]
    
    newfinal4=pd.merge_asof(
    newfinal2,
    mealtime3,
   # by="component",
    right_on="dt",
    left_on="dt",
    tolerance=pd.Timedelta('120 minute'))
    
    Mergedfinal2=newfinal4[(newfinal4["tm2nomeal_y"].notna())]
    nomealdata=Mergedfinal2[["Sensor Glucose (mg/dL)"]]
    nomealdata = nomealdata.reset_index()
    nomealdata=nomealdata[["Sensor Glucose (mg/dL)"]]

    #Imput missing observations with mean value
    nomealdata['Sensor Glucose (mg/dL)'].fillna(nomealdata['Sensor Glucose (mg/dL)'].mean(), inplace = True)
    seq= cycle([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
    nomealdata['x'] = [next(seq) for count in range(nomealdata.shape[0])]
    nomealdata['y']=0
    nomealdata['y'] = (nomealdata["x"] %24==0).shift(1).cumsum() + 1
    nomealdata['y'].fillna(value=1, inplace = True)
    nomeal_data=nomealdata.pivot(index='y',columns='x', values='Sensor Glucose (mg/dL)').add_prefix('X').reset_index()
    
    
    ##Drop x and y
    meal_data.drop(['y'], axis=1, inplace=True)
    nomeal_data.drop(['y'], axis=1, inplace=True)
    
    
   ##Min max value training
    meal_data['max_value'] = meal_data.max(axis=1)
    meal_data['min_value'] = meal_data.min(axis=1)
    meal_data['std'] = meal_data.std(axis=1)
    meal_data['dev']=(meal_data['max_value']-meal_data['min_value'])/(meal_data['min_value'])
    
    nomeal_data['max_value'] = nomeal_data.max(axis=1)
    nomeal_data['min_value'] = nomeal_data.min(axis=1)
    nomeal_data['std'] = nomeal_data.std(axis=1)
    nomeal_data['dev']=(nomeal_data['max_value']-nomeal_data['min_value'])/(nomeal_data['min_value'])
    
    ##Set meal & no meal indicator
    meal_data['label']=1
    nomeal_data['label']=0
    
    #Subset final features and label
    meal=meal_data[["std","dev","label"]]
    nomeal=nomeal_data[["std","dev","label"]]

    finalmeal = meal.append(nomeal, ignore_index=True)
    
    ##Train Model
    y_data = finalmeal['label']
    x_data = finalmeal.drop('label', axis = 1)
    
    x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x_data, y_data, test_size = 0.2)
    model = LogisticRegression()
    model.fit(x_training_data, y_training_data)
    predictions = model.predict(x_test_data)
    pkl_filename = "model.pkl"
    with open(pkl_filename, 'wb') as file:
        pickle.dump(model, file)
    
    ##return(classification_report(y_test_data, predictions))

In [62]:
main('Desktop/Project/CGMData.csv','Desktop/Project2/CGM_patient2.csv','Desktop/Project2/InsulinData.csv','Desktop/Project2/Insulin_patient2.csv')

/Users/Akc/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2909: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/Akc/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2909: DtypeWarning: Columns (13,43) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/Akc/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2909: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/Akc/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

'              precision    recall  f1-score   support\n\n           0       0.60      0.59      0.59       129\n           1       0.54      0.54      0.54       112\n\n    accuracy                           0.57       241\n   macro avg       0.57      0.57      0.57       241\nweighted avg       0.57      0.57      0.57       241\n'